In [1]:
import csv
import json
from typing import TypedDict
from collections import defaultdict

In [18]:
file = "../../data/toflit18_all_flows.csv"
product_classification = "product_sitc_simplEN"

In [3]:
class DataPoint(TypedDict):
    product: str 
    import_dk: int
    import_dkb: int
    export_dk: int
    export_dkb: int

In [19]:
report_dk = defaultdict(dict)
report_others = defaultdict(dict)

with open(file, "r") as muerte:
    reader = csv.DictReader(muerte)
    for i, row in enumerate(reader):
        if row["year"] == "1789":
            # process mirrors flows : France imports from Dunkerque 
            if row["export_import"] == "Imports" and row["partner"] == "Dunkerque":
                if row[product_classification]:
                    value = float(row["value"]) if row["value"] else 0.0
                    if row["customs_region_grouping"] not in report_others[row[product_classification]]:
                        report_others[row[product_classification]][row["customs_region_grouping"]] = {}
                    report_others[row[product_classification]][row["customs_region_grouping"]]["product"] = row[product_classification]
                    report_others[row[product_classification]][row["customs_region_grouping"]]["region"] = row["customs_region_grouping"]
                    report_others[row[product_classification]][row["customs_region_grouping"]]["import"] = report_others[row[product_classification]][row["customs_region_grouping"]].get("import", 0) + value

            if row["customs_region_grouping"] == "Flandre" and row["customs_office"] in ['Dunkerque', 'Dunkerque-Basseville']:
                if row[product_classification]:
                    value = float(row["value"]) if row["value"] else 0.0
                    report_dk[row[product_classification]]["product"] = row[product_classification]
                    if row["customs_office"] == 'Dunkerque':
                        if row["export_import"] == "Exports":
                            report_dk[row[product_classification]]["export_dk"] = report_dk[row[product_classification]].get("export_dk", 0) + value
                        elif row["export_import"] == "Imports":
                            report_dk[row[product_classification]]["import_dk"] = report_dk[row[product_classification]].get("import_dk", 0) + value
                    elif row["customs_office"] == 'Dunkerque-Basseville':
                        if row["export_import"] == "Exports":
                            report_dk[row[product_classification]]["export_dkb"] = report_dk[row[product_classification]].get("export_dkb", 0) + value
                        elif row["export_import"] == "Imports":
                            report_dk[row[product_classification]]["import_dkb"] = report_dk[row[product_classification]].get("import_dkb", 0) + value

In [16]:
all_data = []
for (product, flows) in report_dk.items():
    export_to_fr = sum([dp['import'] for (region,dp) in report_others[product].items()])
    export_to_colonies = flows.get("export_dk", 0)
    total = flows.get("import_dk",0)

    all_data.append({"value":total - export_to_fr - export_to_colonies, "product": product, "destination" : "entrepôt"})
    all_data.append({"value":export_to_fr, "product": product, "destination" : "France"})
    all_data.append({"value":export_to_colonies, "product": product, "destination" : "colonies ou re-exp"})

In [21]:
report_others

defaultdict(dict,
            {'Other foodstuffs and live animals': {'Amiens': {'product': 'Other foodstuffs and live animals',
               'region': 'Amiens',
               'import': 6620.0},
              'Caen': {'product': 'Other foodstuffs and live animals',
               'region': 'Caen',
               'import': 4019.400000000001},
              'Marseille': {'product': 'Other foodstuffs and live animals',
               'region': 'Marseille',
               'import': 66960.0},
              'Montpellier': {'product': 'Other foodstuffs and live animals',
               'region': 'Montpellier',
               'import': 10496.99985}},
             'Crude materials, inedible, except combustibles': {'Amiens': {'product': 'Crude materials, inedible, except combustibles',
               'region': 'Amiens',
               'import': 750.0},
              'Caen': {'product': 'Crude materials, inedible, except combustibles',
               'region': 'Caen',
               'import': 0

In [ ]:
with open("data/report_dunkerque.json", "w") as f:
    json.dump(report_dk, f)

In [ ]:
with open("data/report_other_regions.json", "w") as f:
    json.dump(report_others, f)

In [ ]:
with open("data/export_dunkerque_entreprôt.json", "w") as f:
    json.dump(all_data, f)